In [1]:
import numpy as np
import pickle
import networkx as nx
import scipy.sparse as sp

from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
EPS = 1e-15
from torch.nn import Parameter
import pandas as pd
import numpy as np
device = 'cpu'
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import copy
import matplotlib.pyplot as plt
import pandas as pd

C:\Users\ZHANGSHANFAN\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#with open('data/dblp/adj_time_list.pickle', 'rb') as handle:
#with open('data/fb/adj_time_list.pickle', 'rb') as handle:
with open('data/enron10/adj_time_list.pickle', 'rb') as handle:
    adj_time_list = pickle.load(handle,encoding='iso-8859-1')

C:\Users\ZHANGSHANFAN\AppData\Local\Temp\ipykernel_5468\2292249624.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  adj_time_list = pickle.load(handle,encoding='iso-8859-1')


In [3]:
def predict_auc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    return average_precision_score(true_edges, predict_edges), roc_auc_score(true_edges, predict_edges)

In [8]:
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, ne, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        NE = ne
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, cc):
        CC = cc
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=0.4, momentum=0.9)
        #self.loss_function = torch.nn.L1Loss(reduction='sum')
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, cc, edge_train, edge_test):
        CC = cc
        GP = g
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        ap, auc = predict_auc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach()
    
####################################### 以最大化预测效果为目标的类型质心更新 ########################################
class ClusteringLayer(nn.Module):
    """给定节点嵌入，更新类簇质心"""
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, x):
        NE = x
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(ne, ini_cc, g, edge_train, edge_test):        
    NE = ne
    CC = ini_cc
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=0.4, momentum=0.9)
    #loss_function = torch.nn.L1Loss(reduction='sum')
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(5):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    ap, auc = predict_auc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach(), graph_recons.detach(), ap, auc

In [12]:
def get_graph(graph_np, nodes_number):

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # 只取左上角矩阵
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # 只取左上角矩阵
    
    #positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    #choose_positive = posi_edge[positive_index]
    choose_positive = posi_edge
    #posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    #test_positive = posi_edge[posi_not_choose]
    test_positive = posi_edge
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),posi_edge_number*6,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(range(nega_edge.shape[0]),posi_edge_number,replace=False)
    test_negative = nega_edge[test_nega_index]
    #test_negative = nega_edge
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [13]:
embedding_dim = 8
n_clusters = 12
beta = 4.5
LR = 0.01
MO = 0.95
NUM_EPOCH = 70
alpha = 1.0
nodes_number = 184

In [14]:
graph = adj_time_list[0].toarray()
train_edge, test_edge, train_mask, test_mask = get_graph(graph, nodes_number)
    
ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
raw_nodes_embedding = ini_embedding.weight.detach()   # t=0时刻的节点嵌入

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
cluster_centers = kmeans.cluster_centers_
raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)    # t=0时刻的类簇质心

for module_epoch in range(100):
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers, LINK_PROB, ap, auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

    if module_epoch%20==0 or module_epoch==99:
        print("######################### module_epoch ： %d ##########################"%module_epoch)
        print("ap : ",ap)
        print("auc : ",auc)

######################### module_epoch ： 0 ##########################
ap :  0.9108414726912174
auc :  0.8933837429111531
######################### module_epoch ： 20 ##########################
ap :  0.9950017032600093
auc :  0.9958412098298677
######################### module_epoch ： 40 ##########################
ap :  0.9945272718229997
auc :  0.9956143667296786
######################### module_epoch ： 60 ##########################
ap :  0.9944685599202098
auc :  0.9956143667296786
######################### module_epoch ： 80 ##########################
ap :  0.9950073478872224
auc :  0.995992438563327
######################### module_epoch ： 99 ##########################
ap :  0.9946179324903636
auc :  0.995765595463138


In [15]:
pd.DataFrame(raw_cluster_centers.numpy()).to_excel("Enron/Enron_cluster_centers.xlsx")
pd.DataFrame(raw_nodes_embedding.numpy()).to_excel("Enron/Enron_0_nodes_embedding.xlsx")
pd.DataFrame(LINK_PROB.numpy()).to_excel("Enron/Enron_0_link_prob.xlsx")

In [20]:
############################################# TIME REGULAR TO GET NODES EMBEDDING ##########################################
def computer_clustering(nodes_embedding, CC):
    norm_squared = torch.sum((nodes_embedding.unsqueeze(1) - CC)**2, 2)   
    numerator = 1.0 / (1.0 + (norm_squared / alpha))
    power = float(alpha + 1) / 2    
    numerator = numerator**power    
    soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
    return soft_assignments

class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, CC):
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar, soft_assignments

class time_regular(nn.Module):
    def __init__(self, NE):
        super(time_regular, self).__init__()
        self.reconstruction_module = reconstruction_graph(NE)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=0.4, momentum=0.95)
        #self.loss_function = torch.nn.L1Loss(reduction='sum')
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, CC, edge_train, edge_test, history_clustering):
        self.reconstruction_module.train()
        for epoch in range(100):
            self.optimizer.zero_grad()
            graph_reconstruction, clustering = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)+ETA*self.loss_function(clustering, history_clustering)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
            with torch.no_grad():
                if epoch%20==0 or epoch==99:
                    recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
                    ap, auc = predict_auc(recons_test_edges, test_edge)
                    print("######################### module_epoch ： %d ##########################"%epoch)
                    print("ap : ",ap)
                    print("auc : ",auc)
                
        return self.reconstruction_module.nodes_embedding.detach(), graph_reconstruction.detach()

In [21]:
ETA = 2.0
CLUSTER_CENTERS = copy.deepcopy(raw_cluster_centers)
MEMORY = torch.zeros((nodes_number, nodes_number, len(adj_time_list)-1))
MEMORY[:,:,0] = LINK_PROB   # store module

for i in range(1, len(adj_time_list)):
    print("NOW GET THE NODES EMBEDDING OF TIME %d"%i)
    
    #NEWLY_EMBEDDING = copy.deepcopy(raw_nodes_embedding)
    HISTORY_CLUSTERING = computer_clustering(raw_nodes_embedding, CLUSTER_CENTERS)
    
    graph = adj_time_list[i].toarray()
    train_edge, test_edge, train_mask, test_mask = get_graph(graph, nodes_number)
    
    #raw_nodes_embedding = NEWLY_EMBEDDING

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)    # t=0时刻的类簇质心

    update_nodes_module = time_regular(raw_nodes_embedding)
    raw_nodes_embedding, link_prob = update_nodes_module(train_edge, CLUSTER_CENTERS, train_mask, test_mask, HISTORY_CLUSTERING)
            
    if i!=len(adj_time_list)-1:
        MEMORY[:,:,i] = link_prob
    pd.DataFrame(raw_nodes_embedding.numpy()).to_excel("Enron/Enron_%d_nodes_embedding.xlsx"%i)
    pd.DataFrame(link_prob.numpy()).to_excel("Enron/Enron_%d_link_prob.xlsx"%i)

NOW GET THE NODES EMBEDDING OF TIME 1
######################### module_epoch ： 0 ##########################
ap :  0.9419176474341062
auc :  0.945266272189349
######################### module_epoch ： 20 ##########################
ap :  0.964812993037447
auc :  0.9707429322813939
######################### module_epoch ： 40 ##########################
ap :  0.9601272369821501
auc :  0.9721400394477319
######################### module_epoch ： 60 ##########################
ap :  0.9656841139284813
auc :  0.9737836949375411
######################### module_epoch ： 80 ##########################
ap :  0.9671351511410957
auc :  0.9752218934911243
######################### module_epoch ： 99 ##########################
ap :  0.9666176686089288
auc :  0.9759204470742932
NOW GET THE NODES EMBEDDING OF TIME 2
######################### module_epoch ： 0 ##########################
ap :  0.9202412518219426
auc :  0.8948691705233179
######################### module_epoch ： 20 ##########################
ap 

In [22]:
# LINK DETECTION
def get_detection_graph(graph_np, nodes_number):
    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # 只取左上角矩阵
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # 只取左上角矩阵
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*4,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [27]:
def computer_clustering(nodes_embedding, CC):
    norm_squared = torch.sum((nodes_embedding.unsqueeze(1) - CC)**2, 2)   
    numerator = 1.0 / (1.0 + (norm_squared / alpha))
    power = float(alpha + 1) / 2    
    numerator = numerator**power    
    soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
    return soft_assignments

class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        self.linear = torch.nn.Linear(w+1, 1)
        
    def forward(self, CC):
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        link_prob = torch.exp(-distance_similar)
    
        return link_prob, soft_assignments

class link_detection(nn.Module):
    def __init__(self, NE):
        super(link_detection, self).__init__()
        self.reconstruction_module = reconstruction_graph(NE)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=0.1, momentum=0.95)
        self.loss_function = torch.nn.MSELoss(reduction='mean')
        #self.cluster_loss_function = torch.nn.MSELoss(reduction='mean')
        #self.linear = torch.nn.Linear(w+1, 1)
        #self.linear_optimizer = torch.optim.SGD(params=self.linear.parameters(), lr=0.1, momentum=0.9)
        
    def forward(self, g, CC, edge_train, edge_test, history_clustering, history_link_prob):
        self.reconstruction_module.train()
        for epoch in range(200):
            self.optimizer.zero_grad()
            #self.linear_optimizer.zero_grad()
            link_prob, clustering = self.reconstruction_module(CC)
            link_prob = link_prob.unsqueeze(2)
            #graph_reconstruction = self.linear(torch.cat((link_prob.long(), history_link_prob), dim=2))
            graph_reconstruction = self.reconstruction_module.linear(torch.cat((link_prob.long(), history_link_prob), dim=2))
            graph_train = torch.take(graph_reconstruction, edge_train.long())
            loss = self.loss_function(g, graph_train)+ETA*self.loss_function(clustering, history_clustering)
            loss.backward()
            #self.linear_optimizer.step()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
            #print(graph_reconstruction)
            if epoch%20==0 or epoch==99:
                print("######################### module_epoch ： %d ##########################"%epoch)
                with torch.no_grad():
                    recons_test_edges = torch.take(graph_reconstruction, edge_test.long()).detach()
                    predict_edges = np.array(recons_test_edges)
                    ap = average_precision_score(test_edge, predict_edges)
                    auc = roc_auc_score(test_edge, predict_edges)
                    print("ap : ",ap)
                    print("auc : ",auc)
            
        return self.reconstruction_module.nodes_embedding.detach()

In [28]:
T = len(adj_time_list)
w = T-3

for i in range(T-3, T):
    print("NOW GET THE NODES EMBEDDING OF TIME %d"%i)
    HISTORY_LINK_PROB = MEMORY[:,:,i-w:i]    # READ MEMORY

    raw_nodes_embedding = pd.read_excel("Enron/Enron_%d_nodes_embedding.xlsx"%(i-1)).iloc[:, 1:]
    raw_nodes_embedding = torch.from_numpy(np.array(raw_nodes_embedding))
    
    HISTORY_CLUSTERING = computer_clustering(raw_nodes_embedding, CLUSTER_CENTERS)
    
    graph = adj_time_list[i].toarray()
    train_edge, test_edge, train_mask, test_mask = get_detection_graph(graph, nodes_number)

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)    # t=0时刻的类簇质心
    
    update_nodes_module = link_detection(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, CLUSTER_CENTERS, train_mask, test_mask, HISTORY_CLUSTERING, HISTORY_LINK_PROB)

NOW GET THE NODES EMBEDDING OF TIME 8
######################### module_epoch ： 0 ##########################
ap :  0.43302528204443747
auc :  0.1488
######################### module_epoch ： 20 ##########################
ap :  0.9732444444444445
auc :  0.9664
######################### module_epoch ： 40 ##########################
ap :  0.9815029415029415
auc :  0.9808
######################### module_epoch ： 60 ##########################
ap :  0.9719865659244971
auc :  0.9712
######################### module_epoch ： 80 ##########################
ap :  0.9663002750986182
auc :  0.9663999999999999
######################### module_epoch ： 99 ##########################
ap :  0.9693026591879587
auc :  0.9696
######################### module_epoch ： 100 ##########################
ap :  0.9693026591879587
auc :  0.9696
######################### module_epoch ： 120 ##########################
ap :  0.9703754561228246
auc :  0.9696
######################### module_epoch ： 140 #######################

In [49]:
def get_next_time_graph(graph_np, nodes_number):

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # 只取左上角矩阵
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # 只取左上角矩阵
    
    test_positive = posi_edge
    
    test_nega_index = np.random.choice(range(nega_edge.shape[0]),posi_edge_number,replace=False)
    test_negative = nega_edge[test_nega_index]

    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return test_edge, test_mask

In [51]:
############################################## LINK PREDICTION ##############################################
predict_w = T-4

class predict_next_graph(nn.Module):
    def __init__(self):
        super(predict_next_graph, self).__init__()
        self.linear = torch.nn.Linear(predict_w, 1)
        
    def forward(self, history_link_prob):
        predicted_graph = self.linear(history_link_prob)
        return predicted_graph

predict_loss_function = torch.nn.MSELoss(reduction='mean')
    
for i in range(T-4, T-1):
    print("NOW GET THE NODES EMBEDDING OF TIME %d"%(i+1))
    predict_modle = predict_next_graph()
    print(predict_modle.linear.state_dict())
    predict_optimizer = torch.optim.SGD(params=predict_modle.parameters(), lr=0.4, momentum=0.95)
    
    HISTORY_LINK = MEMORY[:,:,i-predict_w:i]    # READ MEMORY
    
    PREDICT_HISTORY_LINK = MEMORY[:,:,i+1-predict_w:i+1]    # READ MEMORY
    
    predict_graph = adj_time_list[i+1].toarray()
    predict_test_edge, predict_test_mask = get_next_time_graph(predict_graph, nodes_number)
    
    train_graph = adj_time_list[i].toarray()
    train_edge, test_edge, train_mask, test_mask = get_graph(train_graph, nodes_number)
    
    predict_modle.train()
    
    for epoch in range(100):
        predict_optimizer.zero_grad()
        predicted_link = predict_modle(HISTORY_LINK)
        graph_train = torch.take(predicted_link, train_mask.long())
        loss = predict_loss_function(train_edge, graph_train)
        loss.backward()
        predict_optimizer.step()
        if epoch%20==0 or epoch==99:
            print("######################### module_epoch ： %d ##########################"%epoch)
            with torch.no_grad():
                recons_test_edges = torch.take(predicted_link, test_mask.long()).detach()
                predict_edges = np.array(recons_test_edges)
                ap = average_precision_score(test_edge, predict_edges)
                auc = roc_auc_score(test_edge, predict_edges)
                print("ap : ",ap)
                print("auc : ",auc)
    
    with torch.no_grad():
        print("################## PREDICT NEXT GRAPH ##################")
        next_graph_predict = predict_modle(PREDICT_HISTORY_LINK)
        linear_test_edges = torch.take(next_graph_predict, predict_test_mask.long()).detach()
        linear_test_edges = np.array(linear_test_edges)
        print("ap : ", average_precision_score(predict_test_edge, linear_test_edges))
        print("auc : ", roc_auc_score(predict_test_edge, linear_test_edges))
        
    print(predict_modle.linear.state_dict())

NOW GET THE NODES EMBEDDING OF TIME 8
OrderedDict([('weight', tensor([[-0.2513, -0.3604, -0.1711, -0.3273,  0.2632,  0.3320,  0.2982]])), ('bias', tensor([-0.0810]))])
######################### module_epoch ： 0 ##########################
ap :  0.5290354487611812
auc :  0.34045212392007373
######################### module_epoch ： 20 ##########################
ap :  0.94230853260999
auc :  0.9458826102579325
######################### module_epoch ： 40 ##########################
ap :  0.944495723235767
auc :  0.9418206814666689
######################### module_epoch ： 60 ##########################
ap :  0.9465550186308131
auc :  0.9486478463966005
######################### module_epoch ： 80 ##########################
ap :  0.947892644266608
auc :  0.9477104782140011
######################### module_epoch ： 99 ##########################
ap :  0.9457744419433296
auc :  0.9479916886687809
################## PREDICT NEXT GRAPH ##################
ap :  0.9480572526680113
auc :  0.9571012078300

In [70]:
def get_new_link(graph_index, nodes_number):
    old_graph = adj_time_list[graph_index].toarray()
    new_graph = adj_time_list[graph_index+1].toarray()
    graph_np = new_graph - old_graph
    graph_np[graph_np!=1]=0

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # 只取左上角矩阵
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # 只取左上角矩阵
    
    test_positive = posi_edge
    
    test_nega_index = np.random.choice(range(nega_edge.shape[0]),posi_edge_number,replace=False)
    test_negative = nega_edge[test_nega_index]

    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return test_edge, test_mask

In [75]:
########################################### NEW LINK PREDICTION ##############################################
predict_w = T-4

class predict_next_graph(nn.Module):
    def __init__(self):
        super(predict_next_graph, self).__init__()
        self.linear = torch.nn.Linear(predict_w, 1)
        
    def forward(self, history_link_prob):
        predicted_graph = self.linear(history_link_prob)
        return predicted_graph

predict_loss_function = torch.nn.MSELoss(reduction='mean')
    
for i in range(T-4, T-1):
    print("NOW GET THE NODES EMBEDDING OF TIME %d"%(i+1))
    predict_modle = predict_next_graph()
    print(predict_modle.linear.state_dict())
    predict_optimizer = torch.optim.SGD(params=predict_modle.parameters(), lr=0.4, momentum=0.95)
    
    HISTORY_LINK = MEMORY[:,:,i-predict_w:i]    # READ MEMORY
    
    PREDICT_HISTORY_LINK = MEMORY[:,:,i+1-predict_w:i+1]    # READ MEMORY
    
    predict_graph = adj_time_list[i+1].toarray()
    predict_test_edge, predict_test_mask = get_new_link(i, nodes_number)
    
    train_graph = adj_time_list[i].toarray()
    train_edge, test_edge, train_mask, test_mask = get_graph(train_graph, nodes_number)
    
    predict_modle.train()
    
    for epoch in range(100):
        predict_optimizer.zero_grad()
        predicted_link = predict_modle(HISTORY_LINK)
        graph_train = torch.take(predicted_link, train_mask.long())
        loss = predict_loss_function(train_edge, graph_train)
        loss.backward()
        predict_optimizer.step()
        if epoch%20==0 or epoch==99:
            print("######################### module_epoch ： %d ##########################"%epoch)
            with torch.no_grad():
                recons_test_edges = torch.take(predicted_link, test_mask.long()).detach()
                predict_edges = np.array(recons_test_edges)
                ap = average_precision_score(test_edge, predict_edges)
                auc = roc_auc_score(test_edge, predict_edges)
                print("ap : ",ap)
                print("auc : ",auc)
    
    with torch.no_grad():
        print("################## PREDICT NEXT GRAPH ##################")
        next_graph_predict = predict_modle(PREDICT_HISTORY_LINK)
        linear_test_edges = torch.take(next_graph_predict, predict_test_mask.long()).detach()
        linear_test_edges = np.array(linear_test_edges)
        print("ap : ", average_precision_score(predict_test_edge, linear_test_edges))
        print("auc : ", roc_auc_score(predict_test_edge, linear_test_edges))
        
    print(predict_modle.linear.state_dict())

NOW GET THE NODES EMBEDDING OF TIME 8
OrderedDict([('weight', tensor([[-0.0135,  0.3022, -0.0173,  0.0136, -0.1330, -0.2126, -0.1314]])), ('bias', tensor([0.2442]))])
######################### module_epoch ： 0 ##########################
ap :  0.3724969562428556
auc :  0.16880438688309457
######################### module_epoch ： 20 ##########################
ap :  0.9538478583856305
auc :  0.9461638207127122
######################### module_epoch ： 40 ##########################
ap :  0.9362949384117102
auc :  0.9279320095611554
######################### module_epoch ： 60 ##########################
ap :  0.9548164537710752
auc :  0.9471949257135716
######################### module_epoch ： 80 ##########################
ap :  0.9516309148135685
auc :  0.9448202596509866
######################### module_epoch ： 99 ##########################
ap :  0.9525351482608914
auc :  0.9458201190457592
################## PREDICT NEXT GRAPH ##################
ap :  0.8724061639179441
auc :  0.9068040876